In [ ]:
!git clone https://github.com/Jawakar-7/Image-Classification-using-multiple-models-.git

In [ ]:
import seaborn as sns
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import confusion_matrix,classification_report

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from  keras.models import Sequential
from keras.layers import Activation , BatchNormalization , Dense , Dropout , Flatten , MaxPooling2D,Conv2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from  keras.models import Sequential

In [1]:
train_path='/content/Image-Classification-using-multiple-models-/Datasets/seg_test/seg_test'
test_path='/content/Image-Classification-using-multiple-models-/Datasets/seg_train/seg_train'

In [ ]:
train-x=